In [5]:

"""
all code below is legacy code written for Python 2.7

The code is meant to remove duplicate reviews and empty reviews (i.e. those without written comment).
It is also meant to identify whether some comments about GP practices have no corresponding 
GP practice data downloaded from the NHS choices database (GP practice data are downloaded in step3)
"""

import csv
import sys
import os


# remove duplicate reviews from dataset

wchod = "revs_with_ods.csv"
wychod = "revs_no_reps.csv"

if os.path.isfile(wychod):
    os.remove(wychod)

nr_porz = 0
typ = 0
inst = 0
ls_porz = {}
tekst = 0
with open(wchod, "rb") as dok:
    reader = csv.reader(dok, delimiter=",")
    for i,v in enumerate(reader):
        if i != 0:
#             dwie opcje dla "reply" i "comment", tak żeby ls_porz szukała tylko powtórek "reply" i "comment" a nie mieszała je
            if v[typ] == "reply":
                alfa = v[nr_porz] + "r"
                if alfa not in ls_porz:
                    ls_porz[alfa] = 1
                else:
                    ls_porz[alfa] = ls_porz[alfa] + 1
                    
            if v[typ] == "comment":
                if v[nr_porz] not in ls_porz:
                    ls_porz[v[nr_porz]] = 1
                else:
                    ls_porz[v[nr_porz]] = ls_porz[v[nr_porz]] + 1
        else:
            nr_porz = v.index("nhs.ref.num")
            typ = v.index('mssg.type')
            inst = v.index('instttn.type')
            tekst = v.index("content")

print "dictionary ready"
# print "słownik zrobiony"

rejestr = []
mini_rejestr = []
for k,v in ls_porz.iteritems():
    if v > 1:
        temp = []
        if k[-1] == "r":
            temp.append(k[:-1])
            mini_rejestr.append(k[:-1])
            temp.append("r")
            rejestr.append(temp)
        else:
            temp.append(k)
            mini_rejestr.append(k)
            temp.append("c")
            rejestr.append(temp)

powtorki = []
with open(wchod, "rb") as dok:
    reader = csv.reader(dok, delimiter=",")
    with open(wychod, "ab") as dok2:
        writer = csv.writer(dok2, delimiter=",")
        for i,v in enumerate(reader):
            if i != 0:
                if v[nr_porz] not in mini_rejestr:
                    writer.writerow(v)
                else:
                    a = v[inst][0:2],v[2],v[typ][0], sys.getsizeof(v[tekst])
                    if a not in powtorki:
                        powtorki.append(a)
                        writer.writerow(v)
                    else:
                        pass
            else:
                writer.writerow(v)
                


print "ok"


słownik zrobiony
[('gp', '1359621', 'c', 133), ('gp', '1382944', 'r', 536), ('gp', '1382942', 'r', 274), ('gp', '1383028', 'r', 566), ('gp', '1382999', 'r', 539), ('gp', '1383086', 'r', 242), ('gp', '1382952', 'r', 285), ('gp', '1383096', 'c', 527), ('gp', '1383093', 'c', 294), ('gp', '1383092', 'c', 148), ('gp', '1383089', 'c', 195), ('gp', '1383086', 'c', 480), ('gp', '1383084', 'c', 1539), ('gp', '1383078', 'c', 345), ('gp', '1383077', 'c', 1091), ('gp', '1383066', 'c', 1127), ('gp', '1383065', 'c', 1409), ('gp', '1383056', 'c', 218), ('gp', '1383049', 'c', 497), ('gp', '1383048', 'c', 483), ('gp', '1383044', 'c', 522), ('gp', '1383038', 'c', 600), ('gp', '1383033', 'c', 303), ('gp', '1356627', 'r', 342), ('gp', '1383028', 'c', 504), ('gp', '1383027', 'c', 209), ('gp', '1359621', 'r', 342), ('gp', '1383025', 'c', 36), ('gp', '1383010', 'c', 773), ('gp', '1383007', 'c', 250), ('gp', '1383004', 'c', 163), ('gp', '1383000', 'c', 473), ('gp', '1382999', 'c', 529), ('gp', '1382995', 'c',

In [4]:
#count how much of which comments are there in the .csv file

import pandas as pd
import os

from_csv = pd.read_csv('final2016_no_reps.csv', sep=',', low_memory=False)
# from_csv.head()

from_csv['mssg.type'].value_counts(normalize=False, sort=True, ascending=False, bins=None, dropna=True)

# os.getcwd()


7678


In [1]:
# -*- coding: utf-8 -*-

# removing reviews with no comment text

import csv
import os
import glob
import sys
import requests


my_dir = "c:\users\user\desktop\gp practices 2016"
adres_przychodni = 'C:\Users\user\Desktop\doktorat\dane na doktorat\przychodnie lista\przychodnie_info' # folder where gp practice data are stored

os.chdir(my_dir) 

wchod = "revs_no_reps.csv"
wychod = "gp_revs_upto12Jan2017.csv"

if os.path.isfile(wychod):
    os.remove(wychod)

os.chdir(adres_przychodni)


kodownik = {}
for plik in glob.glob("*.txt"):
    kod = plik[12:]
    kon = kod.find("_")
    klucz = kod[:kon]
    with open(plik, "rb") as d:
        tekst = d.read()
        test = sys.getsizeof(tekst)
        if test > 2000:
            a = tekst.find("<dt>ODS code:</dt>")
            b = a + len("<dt>ODS code:</dt>") + 4
            c = tekst[b:(b+20)]
            koniec = c.find("</dd>")
            wart = c[:koniec]
            kodownik[klucz] = wart
        else:
            kodownik[klucz] = "nan"

# print str(len(kodownik)) + " przychodni otrzymało komentarze"
print str(len(kodownik)) + " GP practices received feedback"


os.chdir(moje_dir)

braki = []
listka = [8,9,10,11,40,43,46,49,52,55]
with open(wchod, "rb") as wsad:
    reader = csv.reader(wsad, delimiter=",")
    with open(wychod, "ab") as dok:
        writer = csv.writer(dok, delimiter = ",") 
        for i, line in enumerate(reader):
            if i != 0:
                kielbasa = []
                if line[3] == "comment" and line[7] != "nan" and line[1] == "gppractices":
                    kielbasa.append(line[0])
                    kielbasa.append(line[1])
                    
                    a = line[12].find("gppractices/")
                    b = a + len("gppractices/")
                    klucz = line[12][b:]
                    try:
                        kielbasa.append(kodownik[klucz])
                    except KeyError:
                        braki.append(klucz)

                    kielbasa.append(line[7])

                    if len(line[52]) != 0 or len(line[46]) != 0 or len(line[49]) != 0 or len(line[43]) != 0 or len(line[40]) != 0 or len(line[55]) != 0:
                        for pozycja in listka:
                            if len(line[pozycja]) != 0:
                                kielbasa.append(line[pozycja])
                            else:
                                kielbasa.append("nan")
                        writer.writerow(kielbasa)
                    else:
                        pass
                else:
                    pass
            else:
                kielbasa = []
                kielbasa.append(line[0])
                kielbasa.append(line[1])
                kielbasa.append("ODS.kod")
                kielbasa.append(line[7])
                for element in listka:
                    kielbasa.append(line[element])
                writer.writerow(kielbasa)


if len(braki) > 0:
    # this should not happen but may that some comments have no corresponding GP practice data
    
    print "list of GP practices which have comments but data data about them is not available"
    print braki
    print
                
print
# print "plik bez pustaków i z ODS-ami stworzony"
print "ok"

9240 przychodni otrzymało komentarze

plik bez pustaków i z ODS-ami stworzony
